In [1]:
pip install holidays

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.6/184.6 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 45.5 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pymeeus: filename=PyMeeus-0.5.11-py3-none-any.whl size=730984 sha256=4db85f81dbea28e8a1b3645189a3377ed43db71e83d5879fe6dcda4a5c184d42
  Stored in directory: /home/ec2-user/.cache/pip/wheels/a0/8b/b2/810ae5a6f970c8be4725353400d643c90de1c0f023a9884ee7
Successfully built pymeeus
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [88]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import holidays
import time

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/train.csv'
file_key_2 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/test.csv'
file_key_3 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/sample_submission.csv'
file_key_4 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/TPSSEP22_GDP_data_2017_to_2021.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

## Reading data-files
train = pd.read_csv(file_content_stream_1)
train['date'] = pd.to_datetime(train['date'], format = '%Y-%m-%d')

test = pd.read_csv(file_content_stream_2)
test['date'] = pd.to_datetime(test['date'], format = '%Y-%m-%d')

submission = pd.read_csv(file_content_stream_3)
country_gdp = pd.read_csv(file_content_stream_4)

## Basic feature engineering 
train['weekday'] = train['date'].dt.dayofweek
train['month'] = train['date'].dt.month
train['weekend'] = np.where(train['weekday'] >= 5, 1, 0)
train['dayOfMonth'] = train['date'].dt.day
train['dayOfYear'] = train['date'].dt.dayofyear
train['quarter'] = train['date'].dt.quarter
train['year'] = train['date'].dt.year

test['weekday'] = test['date'].dt.dayofweek
test['month'] = test['date'].dt.month
test['weekend'] = np.where(test['weekday'] >= 5, 1, 0)
test['dayOfMonth'] = test['date'].dt.day
test['dayOfYear'] = test['date'].dt.dayofyear
test['quarter'] = test['date'].dt.quarter
test['year'] = test['date'].dt.year

## Appending GDP
train = pd.merge(train, country_gdp, on = ['country', 'year'], how = 'left')
train = train.drop(columns = ['year'], axis = 1)

test = pd.merge(test, country_gdp, on = ['country', 'year'], how = 'left')
test = test.drop(columns = ['year'], axis = 1)

## Extracting holidays
be_holidays = holidays.BE(years = [2017, 2018, 2019, 2020, 2021])
fr_holidays = holidays.FR(years = [2017, 2018, 2019, 2020, 2021])
de_holidays = holidays.DE(years = [2017, 2018, 2019, 2020, 2021])
it_holidays = holidays.IT(years = [2017, 2018, 2019, 2020, 2021])
pl_holidays = holidays.PL(years = [2017, 2018, 2019, 2020, 2021])
es_holidays = holidays.ES(years = [2017, 2018, 2019, 2020, 2021])

train_list = list()
test_list = list()
countries = ['Belgium', 'France', 'Germany', 'Italy', 'Poland', 'Spain']

for i in range(0, len(countries)):
    
    train_temp = train[train['country'] == countries[i]].reset_index(drop = True)
    train_temp['is_holiday'] = np.nan
    train_temp['holiday_season'] = np.nan
     
    test_temp = test[test['country'] == countries[i]].reset_index(drop = True)
    test_temp['is_holiday'] = np.nan
    test_temp['holiday_season'] = np.nan
    
    if (i == 0):
        
        holiday_to_use = be_holidays
        
    elif (i == 1):
        
        holiday_to_use = fr_holidays
        
    elif (i == 2):
        
        holiday_to_use = de_holidays
        
    elif (i == 3):
        
        holiday_to_use = it_holidays
        
    elif (i == 4):
        
        holiday_to_use = pl_holidays
        
    else:
        
        holiday_to_use = es_holidays
    
    for j in range(0, train_temp.shape[0]):
        
        train_temp['is_holiday'][j] = np.where(train_temp['date'][j] in holiday_to_use, 1, 0)
        
        if ((train_temp['date'][j] >= pd.to_datetime('2017-12-08')) & (train_temp['date'][j] <= pd.to_datetime('2017-12-31'))):
            
            train_temp['holiday_season'][j] = 1
            
        elif ((train_temp['date'][j] >= pd.to_datetime('2018-12-08')) & (train_temp['date'][j] <= pd.to_datetime('2018-12-31'))):    
            
            train_temp['holiday_season'][j] = 1
              
        elif ((train_temp['date'][j] >= pd.to_datetime('2019-12-08')) & (train_temp['date'][j] <= pd.to_datetime('2019-12-31'))):      
            
            train_temp['holiday_season'][j] = 1
       
        elif ((train_temp['date'][j] >= pd.to_datetime('2020-12-08')) & (train_temp['date'][j] <= pd.to_datetime('2020-12-31'))):
            
            train_temp['holiday_season'][j] = 1
        
        else:
            
            train_temp['holiday_season'][j] = 0
            
    train_list.append(train_temp)
    
    for k in range(0, test_temp.shape[0]):
        
        test_temp['is_holiday'][k] = np.where(test_temp['date'][k] in holiday_to_use, 1, 0)
        
        if ((test_temp['date'][k] >= pd.to_datetime('2017-12-08')) & (test_temp['date'][k] <= pd.to_datetime('2017-12-31'))):
            
            test_temp['holiday_season'][j] = 1
            
        elif ((test_temp['date'][k] >= pd.to_datetime('2018-12-08')) & (test_temp['date'][k] <= pd.to_datetime('2018-12-31'))):    
            
            test_temp['holiday_season'][j] = 1
              
        elif ((test_temp['date'][k] >= pd.to_datetime('2019-12-08')) & (test_temp['date'][k] <= pd.to_datetime('2019-12-31'))):      
            
            test_temp['holiday_season'][j] = 1
       
        elif ((test_temp['date'][k] >= pd.to_datetime('2020-12-08')) & (test_temp['date'][k] <= pd.to_datetime('2020-12-31'))):
            
            test_temp['holiday_season'][k] = 1
        
        else:
            
            test_temp['holiday_season'][k] = 0
            
    test_list.append(test_temp)
    
## Putting train and test in the right format
train = pd.concat(train_list)
train['is_holiday'] = train['is_holiday'].astype(int)
train['holiday_season'] = train['holiday_season'].astype(int)

test = pd.concat(test_list)
test['is_holiday'] = test['is_holiday'].astype(int)
test['holiday_season'] = test['holiday_season'].astype(int)

/tmp/ipykernel_18409/1084980129.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_temp['is_holiday'][j] = np.where(train_temp['date'][j] in holiday_to_use, 1, 0)
/tmp/ipykernel_18409/1084980129.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_temp['holiday_season'][j] = 0
/tmp/ipykernel_18409/1084980129.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_temp['holiday_season'][j] = 1
/tmp/ipykernel_18409/10

In [89]:
def smape(y_true, y_pred):
    
    smape = abs(y_true - y_pred) / (abs(y_true) + abs(y_pred))
    smape = smape.mean() * 200
    return smape

In [90]:
all_data = pd.concat([train, test], axis = 0)

le = LabelEncoder()
cols = ['country', 'store', 'product']
for col in cols:
    le = LabelEncoder()
    all_data[col] = le.fit_transform(all_data[col])

scaler = MinMaxScaler()    

all_data = all_data.drop(['date', 'row_id'], axis = 1)
train = all_data.iloc[:70128,:]
test = all_data.iloc[70128:,:].drop(['num_sold'], axis = 1)
test = pd.DataFrame(scaler.fit_transform(test), columns = test.columns)

X = train.drop(['num_sold'], axis = 1)
Y = train['num_sold']

X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)

In [86]:
from sklearn.model_selection import GridSearchCV

ridge_model = Ridge()
ridge_params = {'alpha' :  0.0001 * np.arange(1, 1000)}
grid = GridSearchCV(estimator = ridge_model, param_grid = ridge_params, scoring = 'neg_mean_absolute_error', cv = 3)

grid.fit(X, Y)
best_param = grid.best_params_
print(best_param)

{'alpha': 0.0999}


In [111]:
## Importing Ridge and RidgeCV
from sklearn.linear_model import Lasso, LassoCV

## Running cross-validation with different values for lambda (using 4-folds)
lasso_cv = LassoCV(alphas = [0.001, 0.01, 0.1, 1, 10], cv = 4, max_iter = 1000000).fit(X, Y)

## Extracting the optimal alpha
CV_alpha = lasso_cv.alpha_
print(CV_alpha)

1.0


In [98]:
## Importing Ridge and RidgeCV
from sklearn.linear_model import Ridge, RidgeCV

## Running cross-validation with different values for lambda (using 4-folds)
ridge_cv = RidgeCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000], cv = 4).fit(X, Y)

## Extracting the optimal alpha
CV_alpha = ridge_cv.alpha_
print(CV_alpha)

1000.0


In [112]:
t1 = time.time()
kf = KFold(n_splits = 4, shuffle = True, random_state = 888)
score_list_ridge = []
test_preds_ridge = []
fold = 1

for train_index, test_index in kf.split(X, Y):
    
    ## Splitting the data
    X_train , X_val = X.iloc[train_index], X.iloc[test_index]  
    Y_train, Y_val = Y.iloc[train_index], Y.iloc[test_index]    
    
    print("X_train shape is :", X_train.shape, "X_val shape is", X_val.shape)
    y_pred_list = []
    
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
    X_val = pd.DataFrame(scaler.fit_transform(X_val), columns = X_val.columns)

    ridge_md = Lasso(alpha = 1, tol = 1e-2, max_iter = 1000000, random_state = 43).fit(X_train, Y_train)
#     ridge_md = Ridge(alpha = 10000, tol = 1e-2, max_iter = 1000000, random_state = 42).fit(X_train, Y_train)
    result = ridge_md.predict(X_val)
    result = pd.DataFrame({'pred_num_sold': result})
    result['pred_num_sold'] = [0 if i <= 0 else i for i in result['pred_num_sold']]
    
    Y_val = pd.DataFrame(Y_val.reset_index(drop = True))
    
    score = smape(Y_val['num_sold'], result['pred_num_sold'])
    print('Fold ', str(fold), ' result is:', score, '\n')
    score_list_ridge.append(score)

    test_preds_ridge.append(ridge_md.predict(test))
    fold += 1

t2 = time.time()
print('Ridge model with cross validation take : {:.3f} sn.'.format(t2-t1))

X_train shape is : (52596, 12) X_val shape is (17532, 12)
Fold  1  result is: 30.911743604314502 

X_train shape is : (52596, 12) X_val shape is (17532, 12)
Fold  2  result is: 31.11932492942805 

X_train shape is : (52596, 12) X_val shape is (17532, 12)
Fold  3  result is: 31.29495110129774 

X_train shape is : (52596, 12) X_val shape is (17532, 12)
Fold  4  result is: 31.121132722335627 

Ridge model with cross validation take : 0.500 sn.


In [113]:
mean = sum(score_list_ridge) / len(score_list_ridge)
variance = sum([((x - mean) ** 2) for x in score_list_ridge]) / len(score_list_ridge)
res = variance ** 0.5
print("Cross validation mean score:", sum(score_list_ridge) / len(score_list_ridge))
print("Cross validation score's Standart deviation is:", res)

Cross validation mean score: 31.11178808934398
Cross validation score's Standart deviation is: 0.13574849084141238


In [ ]:
test_preds_ridge = pd.DataFrame(np.concatenate(test_preds_ridge, axis = 1))
print(test_preds_ridge.shape)

test_preds_ridge = test_preds_ridge.mean(axis = 1)
print(test_preds_ridge.head())

In [77]:
submission['num_sold'] = np.exp(test_preds_ridge)
submission.to_csv('Ridge_submission_1.csv', index = False)
submission.head()

,row_id,num_sold
0,70128,317.832795
1,70129,316.701413
2,70130,315.574058
3,70131,314.450716
4,70132,110.286642
